<a href="https://colab.research.google.com/github/Anteii/HPC-Labs/blob/main/lab4/MassSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook setup

In [1]:
from IPython.display import clear_output

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [3]:
!nvidia-smi

Thu Nov 17 21:06:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!git clone https://github.com/NVIDIA/cuda-samples.git
!make -C /content/cuda-samples/Samples/1_Utilities/deviceQueryDrv/
!/content/cuda-samples/bin/x86_64/linux/release/deviceQueryDrv

Cloning into 'cuda-samples'...
remote: Enumerating objects: 11024, done.
remote: Counting objects: 100% (11024/11024), done.
remote: Compressing objects: 100% (1850/1850), done.
remote: Total 11024 (delta 9174), reused 10979 (delta 9148), pack-reused 0
Receiving objects: 100% (11024/11024), 127.02 MiB | 16.36 MiB/s, done.
Resolving deltas: 100% (9174/9174), done.
Checking out files: 100% (3615/3615), done.
make: Entering directory '/content/cuda-samples/Samples/1_Utilities/deviceQueryDrv'
/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common  -m64    --threads 0 --std=c++11 -gencode arch=compute_35,code=compute_35 -o deviceQueryDrv.o -c deviceQueryDrv.cpp
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
/usr/local/cuda/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_35,code=compute_35 -o deviceQueryDrv deviceQueryDrv.

In [5]:
!mkdir src

# Python (CPU)

In [17]:
import numpy as np
np.random.seed(41)

def generate_sub(alphabet, h_min, h_max):
  sub_len = np.random.randint(h_min, h_max, size=1)
  return np.random.choice(alphabet, size=sub_len)

def gen_d(alphabet, subs):
  d = {ch: [] for ch in alphabet}
  for sub_ind, sub in enumerate(subs):
    for ch_ind, ch in enumerate(sub):
      d[ch].append((sub_ind, ch_ind))
  return d

def gen_working_mat(subs, N, H):
  mat = np.array([len(sub) for sub in subs])
  return mat.reshape(-1,1).repeat(H, axis=1)

def iterate(text, d, mat):
  for ind, ch in enumerate(text):
    for pair in d[ch]:
      mat[pair[0], ind - pair[1]] -= 1
  return mat

def is_contain(mat):
  return (mat == 0).any(axis=1)

def find_indices(mat):
  sub_ind, pos_ind = np.where(mat == 0)
  return np.stack([sub_ind, pos_ind]).T

In [18]:
# Define alphabet
S = 6
alphabet = np.arange(0, S, dtype=np.uint8)
# Example
#S = 6
#alphabet = ["a", "b", "c", "d", "e", "f"]

In [19]:
# Define text
H = 100
text = np.random.choice(alphabet, size=H)

# Example
#H = 6
#text = np.array(["a", "a", "e", "f", "e", "d"])

In [20]:
h_min, h_max = 1, 20
N = 10
subs = [generate_sub(alphabet, h_min, h_max) for i in range(N)]

# Example
#N = 3
#subs = [np.array(["a", "a"]), np.array(["a"]), np.array(["f", "e", "d"])]

In [21]:
d = gen_d(alphabet, subs)
mat = gen_working_mat(subs, N, H)

In [22]:
mat = iterate(text, d, mat)

In [23]:
is_contain(mat)

array([False, False, False, False, False,  True, False, False, False,
       False])

In [24]:
find_indices(mat)

array([[ 5,  8],
       [ 5, 10],
       [ 5, 18],
       [ 5, 27],
       [ 5, 32],
       [ 5, 34],
       [ 5, 35],
       [ 5, 47],
       [ 5, 49],
       [ 5, 50],
       [ 5, 53],
       [ 5, 56],
       [ 5, 57],
       [ 5, 66],
       [ 5, 67],
       [ 5, 71],
       [ 5, 76],
       [ 5, 77],
       [ 5, 79],
       [ 5, 80],
       [ 5, 82],
       [ 5, 85],
       [ 5, 86],
       [ 5, 95]])

# C++

In [25]:
%%writefile src/cpu.cpp
#include <iostream>
#include <vector>
#include <map>
#include <fstream>
#include <string>
#include <chrono>

using namespace std;
using namespace std::chrono;

enum SearchType{
  Indicies,
  Entries
};

template <typename T>
vector<int> getSubSizes(const vector<vector<T>>& subs){
  vector<int> subsSizes(subs.size(), 0);
  for (int subInd = 0; subInd < subs.size(); ++subInd){
    subsSizes[subInd] = subs[subInd].size();
  }
  return subsSizes;
}

template <typename T>
map<T, vector<pair<int, int>>> generateDict(const vector<T>& alphabet, const vector<vector<T>>& subs){
  map<T, vector<pair<int, int>>> dict;
  for (auto chr : alphabet){
    dict[chr] = vector<pair<int, int>>();
  }
  
  for (int sub_ind = 0; sub_ind < subs.size(); ++sub_ind){
    auto sub = subs[sub_ind];
    for (int chr_ind = 0; chr_ind < sub.size(); ++chr_ind){
      auto chr = sub[chr_ind];
      dict[chr].push_back({sub_ind, chr_ind});
    }
  }
  
  return dict;
}

vector<vector<int>> generateMatrix(const vector<int>& subsSizes, int textSize){
  vector<vector<int>> mat;
  for (auto subSize : subsSizes){
    mat.push_back(vector<int>(textSize, subSize));
  }
  return mat;
}

template <typename T>
vector<T> generateText(const vector<T>& alphabet, int size){
  vector<T> text(size);
  for (int i = 0; i < size; ++i){
      int ind = rand() % alphabet.size();
      text[i] = alphabet[ind];
  }
  return text;
}

template <typename T>
vector<vector<T>> generateSubs(const vector<T>& alphabet, int n, int sizeMin, int sizeMax){
  vector<vector<T>> subs;

  for (int subInd = 0; subInd < n; ++subInd){
    int size = sizeMin + rand() % (sizeMax - sizeMin + 1); 
    auto sub = generateText(alphabet, size);
    subs.push_back(sub);
  }

  return subs;
}

template <typename T>
void iterate(const vector<T>& text, const map<T, vector<pair<int, int>>>& dict,
            vector<vector<int>>& matrix){
  for (int chr_ind = 0; chr_ind < text.size(); ++chr_ind){
    for (const auto pair : dict.at(text[chr_ind])){
      matrix[pair.first][chr_ind - pair.second]--;
    }
  }
}

vector<pair<int, int>> findIndices(const vector<vector<int>>& mat){
  vector<pair<int, int>> result;

  int textSize = mat[0].size();

  for (int subInd = 0; subInd < mat.size(); ++subInd){
    for (int textPos = 0; textPos < textSize; ++textPos){
      if (mat[subInd][textPos] == 0){
        result.push_back({subInd, textPos});
      }
    }
  }

  return result;
}

vector<bool> findEntries(const vector<vector<int>>& mat){
  
  vector<bool> result(mat.size(), false);
  int textSize = mat[0].size();

  for (int subInd = 0; subInd < mat.size(); ++subInd){
    for (int textPos = 0; textPos < textSize; ++textPos){
      if (mat[subInd][textPos] == 0){
        result[subInd] = true;
        break;
      }
    }
  }

  return result;
}

ostream& operator<<(ostream& os, const vector<pair<int, int>>& pairs){
    os << "{";
    for (int i = 0; i < pairs.size(); ++i){
      auto it = pairs[i];
      os << "(" << it.first << ", " << it.second;
      os << (i == pairs.size() - 1 ? ")}" : "), ");
    }

    return os;
}

ostream& operator<<(ostream& os, const vector<bool>& flags){
    os << "{";
    for (int i = 0; i < flags.size(); ++i){
      os << (flags[i] ? "True" : "False");
      os << (i == flags.size() - 1 ? "}" : ", ");
    }

    return os;
}

template<typename T>
void printText(const vector<T>& text, ostream& os){ 
   for (int i = 0; i < text.size(); ++i)
   {
     os << (T)text[i] << (i == text.size() - 1 ? "" : " ");
   }
   os << endl;
}

template<typename T>
void printSubs(const vector<vector<T>>& subs, ostream& os){ 
   for(auto sub : subs){
     printText(sub, os);
   }
}

void search(int textSize, int subN, int subSizeMin, int subSizeMax, SearchType searchType){
  
  ofstream myfile;
  myfile.open ("data.txt");

  // Generate 8bit alphabet
  int alphabetSize = 256;
  vector<int> alphabet(alphabetSize);
  for (int i = 0; i < alphabetSize; ++i){
    alphabet[i] = i;
  }

  // Generate text
  vector<int> text = generateText<int>(alphabet, textSize);

  // Generate substrings
  auto subs = generateSubs<int>(alphabet, subN, subSizeMin, subSizeMax);
  vector<int> subsSizes = getSubSizes<int>(subs);

  // Preliminary step
  map<int, vector<pair<int, int>>> dict = generateDict<int>(alphabet, subs);
  vector<vector<int>> mat = generateMatrix(subsSizes, textSize);
  
  // Algorithm steps
  iterate<int>(text, dict, mat);

  // Save input data
  myfile << textSize << " " << subN << " " <<  subSizeMin << " "
     << subSizeMax << " " << searchType << endl;
  printText(alphabet, myfile);
  printText(text, myfile);
  printSubs(subs, myfile);

  // Interpret results
  switch(searchType){
    case Indicies:{
      vector<pair<int, int>> indices = findIndices(mat);
      myfile << indices << endl;
    }
    break;
    case Entries:{
      vector<bool> entries = findEntries(mat);
      myfile << entries << endl;
      break;
    } 
  }
  myfile.close();
}

int main(int argc, char** argv){
  srand(42);

  int textSize = stoi(argv[1]);
  int subN = stoi(argv[2]);
  int subSizeMin = stoi(argv[3]);
  int subSizeMax = stoi(argv[4]);
  SearchType searchType = (SearchType)stoi(argv[5]);

  auto start = high_resolution_clock::now();

  search(textSize, subN, subSizeMin, subSizeMax, searchType);

  auto end = high_resolution_clock::now();

  auto duration = duration_cast<microseconds>(end - start);

  cout << "Time of execution: " << duration.count() / 1000.0f << "ms" << endl;

  return 0;
}

Overwriting src/cpu.cpp


In [26]:
!g++ -std=c++11 -O2 src/cpu.cpp -o cpu

In [27]:
!./cpu 100 10 1 7 1

Time of execution: 0.337ms


# C++ (GPU) - dev

In [254]:
%%writefile src/gpu.cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <map>
#include <fstream>
#include <string>
#include <chrono>

using namespace std;
using namespace std::chrono;

enum SearchType{
  Indicies,
  Entries
};

enum Device{
  CPU,
  GPU
};

ostream& operator<<(ostream& os, const vector<pair<int, int>>& pairs){
    os << "{";
    for (int i = 0; i < pairs.size(); ++i){
      auto it = pairs[i];
      os << "(" << it.first << ", " << it.second;
      os << (i == pairs.size() - 1 ? ")}" : "), ");
    }

    return os;
}

ostream& operator<<(ostream& os, const vector<bool>& flags){
    os << "{";
    for (int i = 0; i < flags.size(); ++i){
      os << (flags[i] ? "True" : "False");
      os << (i == flags.size() - 1 ? "}" : ", ");
    }

    return os;
}

template<typename T>
void printText(const vector<T>& text, ostream& os){ 
   for (int i = 0; i < text.size(); ++i)
   {
     os << (T)text[i] << (i == text.size() - 1 ? "" : " ");
   }
   os << endl;
}

template<typename T>
void printSubs(const vector<vector<T>>& subs, ostream& os){ 
   for(auto sub : subs){
     printText(sub, os);
   }
}

template <typename T>
vector<int> getSubSizes(const vector<vector<T>>& subs){
  vector<int> subsSizes(subs.size(), 0);
  for (int subInd = 0; subInd < subs.size(); ++subInd){
    subsSizes[subInd] = subs[subInd].size();
  }
  return subsSizes;
}

template <typename T>
map<T, vector<pair<int, int>>> generateDict(const vector<T>& alphabet, const vector<vector<T>>& subs){
  map<T, vector<pair<int, int>>> dict;
  for (auto chr : alphabet){
    dict[chr] = vector<pair<int, int>>();
  }
  
  for (int sub_ind = 0; sub_ind < subs.size(); ++sub_ind){
    auto sub = subs[sub_ind];
    for (int chr_ind = 0; chr_ind < sub.size(); ++chr_ind){
      auto chr = sub[chr_ind];
      dict[chr].push_back({sub_ind, chr_ind});
    }
  }
  
  return dict;
}

vector<vector<int>> generateMatrix(const vector<int>& subsSizes, int textSize){
  vector<vector<int>> mat;
  for (auto subSize : subsSizes){
    mat.push_back(vector<int>(textSize, subSize));
  }
  return mat;
}

template <typename T>
vector<T> generateText(const vector<T>& alphabet, int size){
  vector<T> text(size);
  for (int i = 0; i < size; ++i){
      int ind = rand() % alphabet.size();
      text[i] = alphabet[ind];
  }
  return text;
}

template <typename T>
vector<vector<T>> generateSubs(const vector<T>& alphabet, int n, int sizeMin, int sizeMax){
  vector<vector<T>> subs;

  for (int subInd = 0; subInd < n; ++subInd){
    int size = sizeMin + rand() % (sizeMax - sizeMin + 1); 
    auto sub = generateText(alphabet, size);
    subs.push_back(sub);
  }

  return subs;
}

template <typename T>
void iterateCPU(const vector<T>& text, const map<T, vector<pair<int, int>>>& dict,
            vector<vector<int>>& matrix){
  for (int chr_ind = 0; chr_ind < text.size(); ++chr_ind){
    for (const auto pair : dict.at(text[chr_ind])){
      matrix[pair.first][chr_ind - pair.second]--;
    }
  }

  for (int s = 0; s < matrix.size(); ++s){
      for (int h = 0; h <  matrix[0].size(); ++h){
        printf("%d ", matrix[s][h]);
      }
      printf("\n");
    }
}

void checkError(cudaError_t error){
	if (error != cudaSuccess){
		cout << "Error" << endl;
		cerr << cudaGetErrorString(error) << endl;
		exit(1);
	}
}

// H - length of a text
// N - number of substrings
// C - Alphabet size
template<int blockSize>
__global__ void kernel(int* text, int H, int* matrix, int N, int* dict, int C) {
  int tid = threadIdx.x;
  int i = blockIdx.x * blockSize + tid;

  if (i >= H) return;

  __shared__ int sdata[blockSize];

  int chr = text[i];

  int offset = 3 * chr;
  int pos = dict[offset + 1];
  int n = dict[offset + 2];

  if (i == 0){
    for (int s = 0; s < N; ++s){
      for (int h = 0; h < H; ++h){
        printf("%d ", matrix[s * H + h]);
      }
      printf("\n");
    }
  }

  if (i == 0){
    printf("i\tchr\toffset\tpos\tn\n");
  }
  
  __syncthreads();
  printf("%d\t%d\t%d\t%d\t%d\n", i, chr, offset, pos, n);

  for (int pair_ind = 0; pair_ind < n; ++pair_ind){
    int ind = pos + pair_ind * 2;
    
    printf("i=%d chr=%d ind=%d sub_ind=%d chr_ind=%d\n", i, chr, ind, dict[ind], dict[ind+1]);
    atomicSub(matrix + dict[ind] * H + i - dict[ind + 1], 1);
    printf("%d\n", dict[ind] * H + i - dict[ind + 1]);
  }

  if (i == 0){
    for (int s = 0; s < N; ++s){
      for (int h = 0; h < H; ++h){
        printf("%d ", matrix[s * H + h]);
      }
      printf("\n");
    }
  } 
}

// working if char type is int and alphabet is 0..N
void iterateGPU(const vector<int>& text, const map<int, vector<pair<int, int>>>& dict,
            vector<vector<int>>& matrix){
  
  int textSize = text.size();
  int N = matrix.size();
  int H = matrix[0].size();
  int alphabetSize = dict.size();
  int entryNumber = 0;

  // Flatten map into [(char, offset, n)][(sub_ind, pos_ind)]
  vector<int> dictHost;
  for (auto const& it : dict){
    
    dictHost.push_back(it.first);
    dictHost.push_back(3 * alphabetSize + entryNumber * 2);
    dictHost.push_back(it.second.size());

    entryNumber += it.second.size();
  }

  cout << 2 << endl;

  int dictDeviceSize = 3 * alphabetSize + 2 * entryNumber;

  // Device pointers
  int* textDevice;
  int* matrixDevice;
  int* dictDevice; // [(char, offset, n)][(sub_ind, pos_ind)]
  // Temporary memory for flat pairs
  int* flatPairs = new int[2 * entryNumber];

  // Copy text to a device
  checkError(cudaMalloc(&textDevice, textSize * sizeof(int)));
  checkError(cudaMemcpy(textDevice, &text[0], textSize * sizeof(int), cudaMemcpyHostToDevice));

  cout << 3 << endl;

  // Flatten and copy matrix to a device
  checkError(cudaMalloc(&matrixDevice, N * H * sizeof(int)));
  for (int sub_ind = 0; sub_ind < N; ++sub_ind){
    checkError(cudaMemcpy(matrixDevice + sub_ind * H, &matrix[sub_ind][0], textSize * sizeof(int), cudaMemcpyHostToDevice));
  }

  cout << 4 << endl;

  // Copy meta part of map to a device [(char, offset, n)]
  checkError(cudaMalloc(&dictDevice, dictDeviceSize * sizeof(int)));
  checkError(cudaMemcpy(dictDevice, &dictHost[0], 3 * alphabetSize * sizeof(int), cudaMemcpyHostToDevice));

  cout << 5 << endl;
  
  // Flatten and copy map data to a device [(sub_ind, pos_ind)]
  int curPos = 0;
  for (auto pairs : dict){
    for (auto pair : pairs.second){
      flatPairs[curPos++] = pair.first;
      flatPairs[curPos++] = pair.second;
    }
  }

  checkError(cudaMemcpy(dictDevice + 3 * alphabetSize, flatPairs, 2 * entryNumber * sizeof(int), cudaMemcpyHostToDevice));

  cout << 6 << endl;

  const int blockX = 128;
  const int gridX = int(1.0f * (textSize - 1) / blockX) + 1;

  cout << 7 << endl;

  kernel<blockX><<<gridX, blockX>>>(textDevice, textSize, matrixDevice, N, dictDevice, alphabetSize);
  checkError(cudaDeviceSynchronize());

  cout << 8 << endl;

  for (int i = 0; i < N; ++i){
    checkError(cudaMemcpy(&matrix[i][0], (matrixDevice + i * H), H * sizeof(int), cudaMemcpyDeviceToHost));
  }

  cout << 9 << endl;


  delete[] flatPairs;
  cudaFree(matrixDevice);
  cudaFree(textDevice);
  cudaFree(dictDevice);

}

vector<pair<int, int>> findIndices(const vector<vector<int>>& mat){
  vector<pair<int, int>> result;

  int textSize = mat[0].size();

  for (int subInd = 0; subInd < mat.size(); ++subInd){
    for (int textPos = 0; textPos < textSize; ++textPos){
      if (mat[subInd][textPos] == 0){
        result.push_back({subInd, textPos});
      }
    }
  }

  return result;
}

vector<bool> findEntries(const vector<vector<int>>& mat){
  
  vector<bool> result(mat.size(), false);
  int textSize = mat[0].size();

  for (int subInd = 0; subInd < mat.size(); ++subInd){
    for (int textPos = 0; textPos < textSize; ++textPos){
      if (mat[subInd][textPos] == 0){
        result[subInd] = true;
        break;
      }
    }
  }

  return result;
}

float search(int textSize, int subN, int subSizeMin, 
            int subSizeMax, SearchType searchType, Device device){
  
  ofstream myfile;
  myfile.open ("data.txt");

  float totalTime = 0.0f;
  

  // Generate 8bit alphabet
  /*
  int alphabetSize = 256;
  vector<int> alphabet(alphabetSize);
  for (int i = 0; i < alphabetSize; ++i){
    alphabet[i] = i;
  }

  // Generate text
  vector<int> text = generateText<int>(alphabet, textSize);
  
  // Generate substrings
  auto subs = generateSubs<int>(alphabet, subN, subSizeMin, subSizeMax);
  vector<int> subsSizes = getSubSizes<int>(subs);
  */

  int alphabetSize = 7;
  textSize = 11;
  vector<int> alphabet = {0, 1, 2, 3, 4, 5, 6};
  vector<int> text = {0, 1, 2, 0, 1, 2, 4, 5, 6, 6, 0};
  vector<vector<int>> subs = {{0, 1, 2}, {2, 4}, {6}};
  vector<int> subsSizes = getSubSizes<int>(subs);
  
  // Preliminary step
  map<int, vector<pair<int, int>>> dict = generateDict<int>(alphabet, subs);
  vector<vector<int>> mat = generateMatrix(subsSizes, textSize);
  
  // Algorithm steps
  if (device == CPU){
    auto start = high_resolution_clock::now();
    iterateCPU<int>(text, dict, mat);
    auto end = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(end - start);
    totalTime = duration.count() / 1000.0f;
  }
  else{
    cudaEvent_t startEvent, stopEvent;
    checkError(cudaEventCreate(&startEvent));
    checkError(cudaEventCreate(&stopEvent));
    

    checkError(cudaEventRecord(startEvent, 0));
    iterateGPU(text, dict, mat);
    checkError(cudaEventRecord(stopEvent, 0));

    checkError(cudaDeviceSynchronize());
    checkError(cudaEventElapsedTime(&totalTime, startEvent, stopEvent));

  }

  // Save input data
  myfile << textSize << " " << subN << " " <<  subSizeMin << " "
     << subSizeMax << " " << searchType << endl;
  printText(alphabet, myfile);
  printText(text, myfile);
  printSubs(subs, myfile);

  // Interpret results
  switch(searchType){
    case Indicies:{
      vector<pair<int, int>> indices = findIndices(mat);
      myfile << indices << endl;
    }
    break;
    case Entries:{
      vector<bool> entries = findEntries(mat);
      myfile << entries << endl;
      break;
    } 
  }
  myfile.close();

  return totalTime;
}

int main(int argc, char** argv){
  srand(42);

  int textSize = stoi(argv[1]);
  int subN = stoi(argv[2]);
  int subSizeMin = stoi(argv[3]);
  int subSizeMax = stoi(argv[4]);
  SearchType searchType = (SearchType)stoi(argv[5]);
  Device device = (Device)stoi(argv[6]);

  
  cout << 1 << endl;
  float duration = search(textSize, subN, subSizeMin, subSizeMax, searchType, device);
  cout << "Time of execution: " << duration << "ms" << endl;

  return 0;
}

Overwriting src/gpu.cu


In [255]:
!nvcc src/gpu.cu -o gpu

src/gpu.cu(343): warning: variable "alphabetSize" was declared but never referenced

src/gpu.cu(153): warning: variable "sdata" was declared but never referenced



In [256]:
# textSize subN subSizeMin subSizeMax searchType(0 - inds, 1 - entry) device (0 - CPU, 1 - GPU)
!./gpu 1000 10 1 7 0 1

1
2
3
4
5







6
7
3 3 3 3 3 3 3 3 3 3 3 
2 2 2 2 2 2 2 2 2 2 2 
1 1 1 1 1 1 1 1 1 1 1 
i	chr	offset	pos	n
0	0	0	21	1
1	1	3	23	1
2	2	6	25	2
3	0	0	21	1
4	1	3	23	1
5	2	6	25	2
6	4	12	29	1
7	5	15	31	0
8	6	18	31	1
9	6	18	31	1
10	0	0	21	1
i=0 chr=0 ind=21 sub_ind=0 chr_ind=0
i=1 chr=1 ind=23 sub_ind=0 chr_ind=1
i=2 chr=2 ind=25 sub_ind=0 chr_ind=2
i=3 chr=0 ind=21 sub_ind=0 chr_ind=0
i=4 chr=1 ind=23 sub_ind=0 chr_ind=1
i=5 chr=2 ind=25 sub_ind=0 chr_ind=2
i=6 chr=4 ind=29 sub_ind=1 chr_ind=1
i=8 chr=6 ind=31 sub_ind=2 chr_ind=0
i=9 chr=6 ind=31 sub_ind=2 chr_ind=0
i=10 chr=0 ind=21 sub_ind=0 chr_ind=0
0
0
0
3
3
3
16
30
31
10
i=2 chr=2 ind=27 sub_ind=1 chr_ind=0
i=5 chr=2 ind=27 sub_ind=1 chr_ind=0
13
16
0 3 3 0 3 3 3 3 3 3 2 
2 2 1 2 2 0 2 2 2 2 2 
1 1 1 1 1 1 1 1 0 0 1 
8
9
Time of execution: 5.01981ms


In [257]:
!./gpu 1000 10 1 7 0 0

1
0 3 3 0 3 3 3 3 3 3 2 
2 2 1 2 2 0 2 2 2 2 2 
1 1 1 1 1 1 1 1 0 0 1 
Time of execution: 0.03ms


# C++ (CUDA) - prod

In [277]:
%%writefile src/gpu.cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <map>
#include <fstream>
#include <string>
#include <chrono>

using namespace std;
using namespace std::chrono;

enum SearchType{
  Indicies,
  Entries
};

enum Device{
  CPU,
  GPU
};

ostream& operator<<(ostream& os, const vector<pair<int, int>>& pairs){
    os << "{";
    for (int i = 0; i < pairs.size(); ++i){
      auto it = pairs[i];
      os << "(" << it.first << ", " << it.second;
      os << (i == pairs.size() - 1 ? ")}" : "), ");
    }

    return os;
}

ostream& operator<<(ostream& os, const vector<bool>& flags){
    os << "{";
    for (int i = 0; i < flags.size(); ++i){
      os << (flags[i] ? "True" : "False");
      os << (i == flags.size() - 1 ? "}" : ", ");
    }

    return os;
}

template<typename T>
void printText(const vector<T>& text, ostream& os){ 
   for (int i = 0; i < text.size(); ++i)
   {
     os << (T)text[i] << (i == text.size() - 1 ? "" : " ");
   }
   os << endl;
}

template<typename T>
void printSubs(const vector<vector<T>>& subs, ostream& os){ 
   for(auto sub : subs){
     printText(sub, os);
   }
}

template <typename T>
vector<int> getSubSizes(const vector<vector<T>>& subs){
  vector<int> subsSizes(subs.size(), 0);
  for (int subInd = 0; subInd < subs.size(); ++subInd){
    subsSizes[subInd] = subs[subInd].size();
  }
  return subsSizes;
}

template <typename T>
map<T, vector<pair<int, int>>> generateDict(const vector<T>& alphabet, const vector<vector<T>>& subs){
  map<T, vector<pair<int, int>>> dict;
  for (auto chr : alphabet){
    dict[chr] = vector<pair<int, int>>();
  }
  
  for (int sub_ind = 0; sub_ind < subs.size(); ++sub_ind){
    auto sub = subs[sub_ind];
    for (int chr_ind = 0; chr_ind < sub.size(); ++chr_ind){
      auto chr = sub[chr_ind];
      dict[chr].push_back({sub_ind, chr_ind});
    }
  }
  
  return dict;
}

vector<vector<int>> generateMatrix(const vector<int>& subsSizes, int textSize){
  vector<vector<int>> mat;
  for (auto subSize : subsSizes){
    mat.push_back(vector<int>(textSize, subSize));
  }
  return mat;
}

template <typename T>
vector<T> generateText(const vector<T>& alphabet, int size){
  vector<T> text(size);
  for (int i = 0; i < size; ++i){
      int ind = rand() % alphabet.size();
      text[i] = alphabet[ind];
  }
  return text;
}

template <typename T>
vector<vector<T>> generateSubs(const vector<T>& alphabet, int n, int sizeMin, int sizeMax){
  vector<vector<T>> subs;

  for (int subInd = 0; subInd < n; ++subInd){
    int size = sizeMin + rand() % (sizeMax - sizeMin + 1); 
    auto sub = generateText(alphabet, size);
    subs.push_back(sub);
  }

  return subs;
}

template <typename T>
void iterateCPU(const vector<T>& text, const map<T, vector<pair<int, int>>>& dict,
            vector<vector<int>>& matrix){
  for (int chr_ind = 0; chr_ind < text.size(); ++chr_ind){
    for (const auto pair : dict.at(text[chr_ind])){
      matrix[pair.first][chr_ind - pair.second]--;
    }
  }

}

void checkError(cudaError_t error){
	if (error != cudaSuccess){
		cout << "Error" << endl;
		cerr << cudaGetErrorString(error) << endl;
		exit(1);
	}
}

// H - length of a text
// N - number of substrings
// C - Alphabet size
template<int blockSize>
__global__ void kernel(int* text, int H, int* matrix, int N, int* dict, int C) {
  int tid = threadIdx.x;
  int i = blockIdx.x * blockSize + tid;

  if (i >= H) return;

  int chr = text[i];

  int offset = 3 * chr;
  int pos = dict[offset + 1];
  int n = dict[offset + 2];

  for (int pair_ind = 0; pair_ind < n; ++pair_ind){
    int ind = pos + pair_ind * 2;
    atomicSub(matrix + dict[ind] * H + i - dict[ind + 1], 1);
  }

}

// working if char type is int and alphabet is 0..N
void iterateGPU(const vector<int>& text, const map<int, vector<pair<int, int>>>& dict,
            vector<vector<int>>& matrix){
  int textSize = text.size();
  int N = matrix.size();
  int H = matrix[0].size();
  int alphabetSize = dict.size();
  int entryNumber = 0;

  // Flatten map into [(char, offset, n)][(sub_ind, pos_ind)]
  vector<int> dictHost;
  for (auto const& it : dict){
    
    dictHost.push_back(it.first);
    dictHost.push_back(3 * alphabetSize + entryNumber * 2);
    dictHost.push_back(it.second.size());

    entryNumber += it.second.size();
  }

  int dictDeviceSize = 3 * alphabetSize + 2 * entryNumber;

  // Device pointers
  int* textDevice;
  int* matrixDevice;
  int* dictDevice; // [(char, offset, n)][(sub_ind, pos_ind)]
  // Temporary memory for flat pairs
  int* flatPairs = new int[2 * entryNumber];

  // Copy text to a device
  checkError(cudaMalloc(&textDevice, textSize * sizeof(int)));
  checkError(cudaMemcpy(textDevice, &text[0], textSize * sizeof(int), cudaMemcpyHostToDevice));

  // Flatten and copy matrix to a device
  checkError(cudaMalloc(&matrixDevice, N * H * sizeof(int)));
  for (int sub_ind = 0; sub_ind < N; ++sub_ind){
    checkError(cudaMemcpy(matrixDevice + sub_ind * H, &matrix[sub_ind][0], textSize * sizeof(int), cudaMemcpyHostToDevice));
  }

  // Copy meta part of map to a device [(char, offset, n)]
  checkError(cudaMalloc(&dictDevice, dictDeviceSize * sizeof(int)));
  checkError(cudaMemcpy(dictDevice, &dictHost[0], 3 * alphabetSize * sizeof(int), cudaMemcpyHostToDevice));
  
  // Flatten and copy map data to a device [(sub_ind, pos_ind)]
  int curPos = 0;
  for (auto pairs : dict){
    for (auto pair : pairs.second){
      flatPairs[curPos++] = pair.first;
      flatPairs[curPos++] = pair.second;
    }
  }

  checkError(cudaMemcpy(dictDevice + 3 * alphabetSize, flatPairs, 2 * entryNumber * sizeof(int), cudaMemcpyHostToDevice));

  const int blockX = 128;
  const int gridX = int(1.0f * (textSize - 1) / blockX) + 1;

  kernel<blockX><<<gridX, blockX>>>(textDevice, textSize, matrixDevice, N, dictDevice, alphabetSize);
  checkError(cudaDeviceSynchronize());

  for (int i = 0; i < N; ++i){
    checkError(cudaMemcpy(&matrix[i][0], (matrixDevice + i * H), H * sizeof(int), cudaMemcpyDeviceToHost));
  }

  delete[] flatPairs;
  cudaFree(matrixDevice);
  cudaFree(textDevice);
  cudaFree(dictDevice);
}

vector<pair<int, int>> findIndices(const vector<vector<int>>& mat){
  vector<pair<int, int>> result;

  int textSize = mat[0].size();

  for (int subInd = 0; subInd < mat.size(); ++subInd){
    for (int textPos = 0; textPos < textSize; ++textPos){
      if (mat[subInd][textPos] == 0){
        result.push_back({subInd, textPos});
      }
    }
  }

  return result;
}

vector<bool> findEntries(const vector<vector<int>>& mat){
  
  vector<bool> result(mat.size(), false);
  int textSize = mat[0].size();

  for (int subInd = 0; subInd < mat.size(); ++subInd){
    for (int textPos = 0; textPos < textSize; ++textPos){
      if (mat[subInd][textPos] == 0){
        result[subInd] = true;
        break;
      }
    }
  }

  return result;
}

bool checkEquality(const vector<vector<int>>& mat1, const vector<vector<int>>& mat2){
  for (int i = 0; i < mat1.size(); ++i){
    for (int j = 0; j < mat1[0].size(); ++j){
      if (mat1[i][j] != mat2[i][j]){
        return false;
      }
    }
  }
  return true;
}

void experiment(int textSize, int subN, int subSizeMin, 
            int subSizeMax, SearchType searchType){
  ofstream myfile;
  myfile.open ("data.txt");

  float totalTime = 0.0f;

  // Generate 8bit alphabet

  int alphabetSize = 256;
  vector<int> alphabet(alphabetSize);
  for (int i = 0; i < alphabetSize; ++i){
    alphabet[i] = i;
  }

  // Generate text
  vector<int> text = generateText<int>(alphabet, textSize);
  
  // Generate substrings
  auto subs = generateSubs<int>(alphabet, subN, subSizeMin, subSizeMax);
  vector<int> subsSizes = getSubSizes<int>(subs);
  
  // Preliminary step
  map<int, vector<pair<int, int>>> dict = generateDict<int>(alphabet, subs);
  vector<vector<int>> mat1 = generateMatrix(subsSizes, textSize);
  vector<vector<int>> mat2 = generateMatrix(subsSizes, textSize); // copy

  // Save input data
  myfile << textSize << " " << subN << " " <<  subSizeMin << " "
     << subSizeMax << " " << searchType << endl;
  printText(alphabet, myfile);
  printText(text, myfile);
  printSubs(subs, myfile);

  // CPU
  auto start = high_resolution_clock::now();
  iterateCPU<int>(text, dict, mat1);
  auto end = high_resolution_clock::now();
  auto duration = duration_cast<microseconds>(end - start);
  totalTime = duration.count() / 1000.0f;

  cout << "CPU time: " << totalTime << endl;
  
  // CUDA
  cudaEvent_t startEvent, stopEvent;
  checkError(cudaEventCreate(&startEvent));
  checkError(cudaEventCreate(&stopEvent));
  
  checkError(cudaEventRecord(startEvent, 0));
  iterateGPU(text, dict, mat2);
  checkError(cudaEventRecord(stopEvent, 0));

  checkError(cudaDeviceSynchronize());
  checkError(cudaEventElapsedTime(&totalTime, startEvent, stopEvent));

  cout << "GPU time: " << totalTime << endl;

  // Check equality
  if (checkEquality(mat1, mat2)){
    cout << "Correct!" << endl;
  }
  else{
    cout << "Incorect!" << endl;
  }

  // Interpret results
  switch(searchType){
    case Indicies:{
      vector<pair<int, int>> indices = findIndices(mat1);
      myfile << indices << endl;
    }
    break;
    case Entries:{
      vector<bool> entries = findEntries(mat1);
      myfile << entries << endl;
      break;
    } 
  }
  myfile.close();
}

float search(int textSize, int subN, int subSizeMin, 
            int subSizeMax, SearchType searchType, Device device){
  
  ofstream myfile;
  myfile.open ("data.txt");

  float totalTime = 0.0f;
  
  // Generate 8bit alphabet

  int alphabetSize = 256;
  vector<int> alphabet(alphabetSize);
  for (int i = 0; i < alphabetSize; ++i){
    alphabet[i] = i;
  }

  // Generate text
  vector<int> text = generateText<int>(alphabet, textSize);
  
  // Generate substrings
  auto subs = generateSubs<int>(alphabet, subN, subSizeMin, subSizeMax);
  vector<int> subsSizes = getSubSizes<int>(subs);
  
  // Preliminary step
  map<int, vector<pair<int, int>>> dict = generateDict<int>(alphabet, subs);
  vector<vector<int>> mat = generateMatrix(subsSizes, textSize);
  
  // Algorithm steps
  if (device == CPU){
    auto start = high_resolution_clock::now();
    iterateCPU<int>(text, dict, mat);
    auto end = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(end - start);
    totalTime = duration.count() / 1000.0f;
  }
  else if (device == GPU){
    cudaEvent_t startEvent, stopEvent;
    checkError(cudaEventCreate(&startEvent));
    checkError(cudaEventCreate(&stopEvent));
    
    checkError(cudaEventRecord(startEvent, 0));
    iterateGPU(text, dict, mat);
    checkError(cudaEventRecord(stopEvent, 0));

    checkError(cudaDeviceSynchronize());
    checkError(cudaEventElapsedTime(&totalTime, startEvent, stopEvent));
  }

  // Save input data
  myfile << textSize << " " << subN << " " <<  subSizeMin << " "
     << subSizeMax << " " << searchType << endl;
  printText(alphabet, myfile);
  printText(text, myfile);
  printSubs(subs, myfile);

  // Interpret results
  switch(searchType){
    case Indicies:{
      vector<pair<int, int>> indices = findIndices(mat);
      myfile << indices << endl;
    }
    break;
    case Entries:{
      vector<bool> entries = findEntries(mat);
      myfile << entries << endl;
      break;
    } 
  }
  myfile.close();

  return totalTime;
}

int main(int argc, char** argv){
  srand(42);

  int textSize = stoi(argv[1]);
  int subN = stoi(argv[2]);
  int subSizeMin = stoi(argv[3]);
  int subSizeMax = stoi(argv[4]);
  SearchType searchType = (SearchType)stoi(argv[5]);
  //Device device = (Device)stoi(argv[6]);
  //float duration = search(textSize, subN, subSizeMin, subSizeMax, searchType, device);
  //cout << "Time of execution: " << duration << "ms" << endl;

  experiment(textSize, subN, subSizeMin, subSizeMax, searchType);

  return 0;
}

Overwriting src/gpu.cu


In [278]:
!nvcc src/gpu.cu -o gpu

In [289]:
# textSize subN subSizeMin subSizeMax searchType(0 - inds, 1 - entry) device (0 - CPU, 1 - GPU)
!./gpu 1000000 20 1 20 0 1

CPU time: 390.294
GPU time: 45.3081
Incorect!
